# Optimizing a quantum circuit

In this tutorial we use FunFact to optimize the parameters of a simple quantum circuit. We consider a quantum circuit as a unitary matrix that is factorized as a product of simpler unitary matrices with a Kronecker product structure.

In [ ]:
import sys
sys.path.append('..')
import funfact as ff
import numpy as np
from funfact import active_backend as ab
from funfact.initializers import Normal, Uniform

The matrix that we will attempt to optimize a circuit for is the 3 qubit ($2^3 \times 2^3$) DFT matrix. It is well-known that this matrix can be decomposed in a quantum circuit with $\mathcal{O}(n^2)$ one- and two-qubit gates. This implementation is also known as the Quantum Fourier Transform (QFT).

In [2]:
n = 3
QFT3 = np.matrix(np.fft.fft(np.eye(2**n)) / np.sqrt(2**n))

Next, we need an initializer that will initialize all the leafs (vectorized) matrices in the quantum circuit to randomly generated unitary matrices.

In [3]:
# initializer TODO assume matrix or vectorized matrix (3-way tensor)
# TODO: bumpy numpy version number for batched QR
from numbers import Number
from funfact import active_backend as ab

class Unitary:
    '''Initializes to a unitary. Useful for 2-way tensors that are matrices or
    3-way tensors that are vectors of matrices. 
    '''
    def __init__(self, distribution='normal', dtype=None):
        if distribution == 'normal':
            self.dtype = dtype or ab.float32
            self.distribution = Normal(
                 1.0, truncation=False, dtype=dtype
             )
        elif distribution == 'truncated':
             self.distribution = Normal(
                 1.0, truncation=2, dtype=dtype
             )
        elif distribution == 'uniform':
             self.distribution = Uniform(
                 1.0, dtype=dtype
             )
        else:
             raise ValueError(f'Invalid distribution: {distribution}.')
    
    def __call__(self, shape):
        if len(shape) == 2:
            q, _ = ab.linalg.qr(self.distribution(shape))
        elif len(shape) == 3:
            q, _ = ab.linalg.qr(self.distribution((shape[2], shape[0], shape[1])))
            q = ab.transpose(q, (1, 2, 0))
        else:
            raise TypeError(f'Unitary initializers can only be used for '
                           'matrices or stacks of matrices/')        
        return q

In [ ]:
unitary = Unitary(distribution='normal', dtype=ab.complex64)
Q = unitary((2, 2, 8))

The next step is to formulate a quadratic loss function with a penalty term on the unitariness of the solution/factors.

In [ ]:
# cost function with penalty on unitariness 
# TODO: penalty actually should be on the optimizable factor matrices
from funfact.loss import Loss

class MSE_Unitary(Loss):
    
    def _loss(self, model, target, alpha=1.0):
        if model.ndim == 2:
            return ab.square(ab.abs(ab.subtract(model, target))) + \
                   alpha * ab.square(ab.abs(ab.subtract(ab.matmul(ab.transpose(ab.conjugate(model)), model), ab.eye(*target.shape))))
        elif model.ndim == 3:
            pass # TODO, transpose the above.
        else:
            return ValueError('Unsupported model shape.')
        
mse_unitary = MSE_Unitary()

In [4]:
unitary = Unitary(distribution='normal', dtype=ab.complex64)

# generate gates and circuit as tensor expressions

def single_qubit_gate(i, n):
    #tsrex = (ff.eye(2**i) & ff.tensor(2, 2, initializer=unitary) & ff.eye(2**(n-i-1)))
    tsrex = (ff.tensor(ab.eye(2**i)) & ff.tensor(2, 2, initializer=unitary) & ff.tensor(ab.eye(2**(n-i-1))))
    return tsrex


def controlled_single_qubit_gate(c, t, n, G=ff.tensor(2, 2, initializer=unitary)):
    E0 = ff.tensor(ab.tensor([[1, 0], [0, 0]]))
    E1 = ff.tensor(ab.tensor([[0, 0], [0, 1]]))
    if c < t:
        #Gup = ff.eye(2**c)
        #Gmid = ff.eye(2**(t-c-1))
        #Gdown = ff.eye(2**(n-t-1))
        #tsrex = Gup & E0 & Gmid & ff.eye(2) & Gdown + \
        #        Gup & E1 & Gmid & G & Gdown
        Gup = ff.tensor(ab.eye(2**c))
        Gmid = ff.tensor(ab.eye(2**(t-c-1)))
        Gdown = ff.tensor(ab.eye(2**(n-t-1)))    
        tsrex = ((Gup & E0 & Gmid & ff.tensor(ab.eye(2)) & Gdown) + \
                (Gup & E1 & Gmid & G & Gdown))
        
    else:
        #Gup = ff.eye(2**t)
        #Gmid = ff.eye(2**(c-t-1))
        #Gdown = ff.eye(2**(n-c-1))
        #tsrex = Gup & ff.eye(2) & Gmid & E0 & Gdown + \
        #        Gup & G & Gmid & E1 & Gdown
        Gup = ff.tensor(ab.eye(2**t))
        Gmid = ff.tensor(ab.eye(2**(c-t-1)))
        Gdown = ff.tensor(ab.eye(2**(n-c-1)))
        tsrex = ((Gup & ff.tensor(ab.eye(2)) & Gmid & E0 & Gdown) + \
                (Gup & G & Gmid & E1 & Gdown))
    return tsrex

def cnot(c, t, n):
    X = ff.tensor(ab.tensor([[0, 1], [1, 0]]))
    tsrex = (controlled_single_qubit_gate(c, t, n, G=X))
    return tsrex

def swap(q1, q2, n):
    tsrex = (cnot(q1, q2, n) @ cnot(q2, q1, n) @ cnot(q1, q2, n))
    return tsrex

def two_qubit_gate(i, n):
    tsrex = (ff.eye(2**i) & ff.tensor(4, 4, initializer=unitary) & ff.eye(2**(n-i-2)))
    return tsrex
    
circuit3 = swap(0, 2, 3) @ \
           single_qubit_gate(2, 3) @ \
           controlled_single_qubit_gate(2, 1, 3) @ \
           single_qubit_gate(1, 3) @ \
           controlled_single_qubit_gate(2, 0, 3) @ \
           controlled_single_qubit_gate(1, 0, 3) @ \
           single_qubit_gate(0, 3)

Using backend "jax".

In [ ]:
fac = ff.factorize(circuit3, QFT3, nvec=8, max_steps=15000, tol=1e-4, loss='mse_loss', dtype=ab.complex64, checkpoint_freq=100)

In [ ]:
fac = ff.Factorization.from_tsrex(circuit3, dtype=ab.complex64)
fac.factors

In [9]:
small_circuit = single_qubit_gate(0, 2)
fac = ff.Factorization.from_tsrex(small_circuit, dtype=ab.complex64)
print(fac())
target = ff.Factorization.from_tsrex(small_circuit, dtype=ab.complex64)
print(target())

[[-0.6569594 -0.37677985j  0.        -0.j          0.17791317-0.62832177j
   0.        +0.j        ]
 [ 0.        -0.j         -0.6569594 -0.37677985j  0.        +0.j
   0.17791317-0.62832177j]
 [-0.3480616 -0.5525345j   0.        -0.j         -0.5238639 +0.5469235j
  -0.        +0.j        ]
 [ 0.        -0.j         -0.3480616 -0.5525345j  -0.        +0.j
  -0.5238639 +0.5469235j ]]
[[-0.5665554 -0.41716188j  0.        -0.j         -0.37396234-0.6042707j
   0.        -0.j        ]
 [ 0.        -0.j         -0.5665554 -0.41716188j  0.        -0.j
  -0.37396234-0.6042707j ]
 [ 0.49562734-0.50925875j  0.        +0.j         -0.6432713 +0.28497535j
  -0.        +0.j        ]
 [ 0.        +0.j          0.49562734-0.50925875j -0.        +0.j
  -0.6432713 +0.28497535j]]


In [6]:
fac = ff.factorize(small_circuit, target(), nvec=2, max_steps=15000, tol=1e-4, loss='mse_loss', dtype=ab.complex64, checkpoint_freq=100)

(4, 4, 8)


  0%|                                                 | 0/15000 [00:00<?, ?it/s]

curr_loss: [0.2623679  0.2623679  0.51667434 0.51667434 0.2623679  0.2623679
 0.51667434 0.51667434]
best_loss: [inf inf]


ValueError: operands could not be broadcast together with shapes (8,) (2,) 

In [15]:
nvec = 3
circuit_vec = ff.vectorize(small_circuit, nvec)
opt_fac = ff.Factorization.from_tsrex(circuit_vec, dtype=ab.complex64)

loss = getattr(ff.loss, 'mse_loss')
ab.to_numpy(loss(opt_fac(), target(), sum_vec=False))

array([0.6323453 , 0.6323453 , 0.6323453 , 0.55357987, 0.55357987,
       0.55357987, 0.5815985 , 0.5815985 , 0.5815985 , 0.6323453 ,
       0.6323453 , 0.6323453 , 0.55357987, 0.55357987, 0.55357987,
       0.5815985 , 0.5815985 , 0.5815985 , 0.6323453 , 0.6323453 ,
       0.6323453 , 0.55357987, 0.55357987, 0.55357987, 0.5815985 ,
       0.5815985 , 0.5815985 ], dtype=float32)

In [16]:
opt_fac().shape nvec**2

(4, 4, 27)

In [ ]:
A = ff.tensor('A', ab.eye(1))
B = ff.tensor('A', ab.eye(2))
tsrex = A & B
fac = ff.Factorization.from_tsrex(tsrex, dtype=ab.complex64)
fac()

In [ ]:
fac = ff.factorize(circuit3, QFT3)